In [1]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/new_project1/'
final_path = '/content/drive/MyDrive/new_project1/final/data/'
final_img = '/content/drive/MyDrive/new_project1/final/img/'
final_result = '/content/drive/MyDrive/new_project1/final/result/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [4]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import re
from io import StringIO

from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv(f'{final_path}final_data.csv')
df.head()

,ADM_CD,행정동코드,total_all,solo_all,total_20,solo_20,total_30,solo_30,total_40,solo_40,...,편의점_근접9,쇼핑_근접3,쇼핑_근접5,쇼핑_근접9,반려동물업_근접3,반려동물업_근접5,반려동물업_근접9,사업체합계2_근접3,사업체합계2_근접5,사업체합계2_근접9
0,11010530,1101053.0,8451.0,2081.0,1273.0,435.0,1535.0,480.0,1659.0,389.0,...,68.0,0.0,7.0,7.0,12.0,34.0,52.0,2297.0,12990.0,17465.0
1,11010540,1101054.0,2569.0,894.0,486.0,242.0,393.0,175.0,415.0,129.0,...,80.0,0.0,7.0,7.0,14.0,42.0,61.0,3698.0,14292.0,21026.0
2,11010550,1101055.0,9202.0,1563.0,1984.0,370.0,1330.0,320.0,1826.0,216.0,...,76.0,0.0,0.0,7.0,9.0,10.0,58.0,2703.0,4214.0,20608.0
3,11010560,1101056.0,15577.0,1633.0,2444.0,260.0,2194.0,247.0,3113.0,226.0,...,76.0,0.0,0.0,7.0,12.0,22.0,61.0,2437.0,5023.0,20342.0
4,11030740,1103074.0,19025.0,5428.0,2988.0,1198.0,4326.0,1544.0,3422.0,919.0,...,109.0,1.0,2.0,12.0,40.0,45.0,73.0,2801.0,3922.0,11388.0


In [6]:
df.isna().sum()

ADM_CD        0
행정동코드         0
total_all     0
solo_all      0
total_20      0
             ..
반려동물업_근접5     0
반려동물업_근접9     0
사업체합계2_근접3    0
사업체합계2_근접5    0
사업체합계2_근접9    0
Length: 583, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Columns: 583 entries, ADM_CD to 사업체합계2_근접9
dtypes: float64(574), int64(2), object(7)
memory usage: 1.9+ MB


In [8]:
df.columns[:24]

Index(['ADM_CD', '행정동코드', 'total_all', 'solo_all', 'total_20', 'solo_20',
       'total_30', 'solo_30', 'total_40', 'solo_40', 'total_50', 'solo_50',
       'total_60', 'solo_60', 'total_70', 'solo_70', '자치구', '행정동명', '시도명',
       '시군구', '읍면동', 'key1', 'key2', '금융기관'],
      dtype='object')

In [9]:
for i in range(0, 16):
    print(df.columns[10 * i + 23 : 10 * i + 9 +23].values)

['금융기관' '유통업체합계' '대형마트' '백화점' '전문점' '쇼핑센터' '복합쇼핑몰' '그밖의 대규모점포' '사업체합계']
['광업' '제조업' '전기 가스 증기 및 공기조절 공급업' '수도 하수 및 폐기물 처리 원료 재생업' '건설업' '도매 및 소매업'
 '운수 및 창고업' '숙박 및 음식점업' '정보통신업']
['부동산업' '전문 과학 및 기술 서비스업' '사업시설 관리 사업 지원 및 임대 서비스업' '공공행정 국방 및 사회보장 행정'
 '교육 서비스업' '보건업 및 사회복지 서비스업' '예술 스포츠 및 여가관련 서비스업'
 '협회 및 단체 수리 및 기타 개인 서비스업' '사업체합계_종사자통계']
['평균종사자' '사업체밀도' '종사자밀도' '인구밀도' '종사자밀도비' '지가 변동률' '지가 누계' '노인복지시설 개수'
 '안심귀갓길 개수']
['동물미용업 동물미용업' '동물병원 동물병원' '동물약국 동물약국' '미용업 기타' '미용업 네일아트업' '미용업 메이크업업'
 '미용업 숙박업 기타' '미용업 일반미용업' '미용업 일반이용업']
['병원 병원' '병원 요양병원(노인병원)' '병원 요양병원(일반요양병원)' '병원 정신병원' '병원 종합병원' '병원 치과병원'
 '병원 한방병원' '세탁업 빨래방업' '세탁업 세탁업 기타']
['세탁업 일반세탁업' '약국 약국' '영화상영관 영화상영관' '의원 보건소' '의원 보건지소' '의원 의원' '의원 조산원'
 '의원 치과의원' '의원 한의원']
['이용업 일반이용업' '일반음식점 감성주점' '일반음식점 경양식' '일반음식점 기타' '일반음식점 김밥(도시락)'
 '일반음식점 까페' '일반음식점 냉면집' '일반음식점 라이브카페' '일반음식점 복어취급']
['일반음식점 뷔페식' '일반음식점 식육(숯불구이)' '일반음식점 외국음식전문점(인도,태국등)' '일반음식점 이동조리'
 '일반음식점 일반음식점' '일반음식점 일식' '일반음식점 전통찻집' '일반음식점 정종/대포집/소주방' '일반음식점 중국식']
['일반음식

In [10]:
### 연령대별로 많이 거주하는 지역만 뺴서

In [11]:
# 비율변수 생성

df['solo_67'] = df.apply(lambda x: (x['solo_60'] + x['solo_70']) / 2, axis = 1)

df['solo_all_ratio'] = df.apply(lambda x: x['solo_all'] / x['total_all'] * 100, axis=1)
df['solo_20_ratio'] = df.apply(lambda x: x['solo_20'] / x['solo_all'] * 100, axis=1)
df['solo_30_ratio'] = df.apply(lambda x: x['solo_30'] / x['solo_all'] * 100, axis = 1)
df['solo_40_ratio'] = df.apply(lambda x: x['solo_40'] / x['solo_all'] * 100, axis=1)
df['solo_50_ratio'] = df.apply(lambda x: x['solo_50'] / x['solo_all'] * 100, axis = 1)
df['solo_60_ratio'] = df.apply(lambda x: x['solo_60'] / x['solo_all'] * 100, axis = 1)
df['solo_70_ratio'] = df.apply(lambda x: x['solo_70'] / x['solo_all'] * 100, axis = 1)
df['solo_67_ratio'] = df.apply(lambda x: (x['solo_60'] + x['solo_70']) / x['solo_all'] * 100, axis = 1)

# 변수선택

In [12]:
# 설명변수 선택
depend_list = ['solo_all_ratio', 'solo_20_ratio', 'solo_30_ratio',  'solo_67_ratio']

selected_var = ['금융기관_근접3', '사업체합계2', '지가 누계', '노인복지시설 개수', '안심귀갓길 개수',
                '대학교 개수_근접3', '반려동물업', '미용업', '병원', '세탁업', '약국', '영화상영관', '이용업',
                '일반음식점', '카페', '편의점', '쇼핑_근접5', '대형마트_근접3']

selected_var_name = ['금융기관', '사업체수', '지가상승률', '노인복지시설', '안심귀갓길',
                     '대학교', '반려동물업', '미용업', '병원', '세탁업', '약국', '영화상영관', '이용업',
                     '일반음식점', '카페', '편의점', '쇼핑', '대형마트']

In [13]:
df = df[depend_list + selected_var]
df.columns = depend_list + selected_var_name
df.head()

,solo_all_ratio,solo_20_ratio,solo_30_ratio,solo_67_ratio,금융기관,사업체수,지가상승률,노인복지시설,안심귀갓길,대학교,...,병원,세탁업,약국,영화상영관,이용업,일반음식점,카페,편의점,쇼핑,대형마트
0,24.624305,20.903412,23.065834,25.036040,5.0,4213.0,4.353,1,0.0,2.0,...,2.0,8.0,14.0,4.0,10.0,1079.0,116.0,13.0,7.0,0.0
1,34.799533,27.069351,19.574944,24.496644,11.0,652.0,4.253,0,0.0,1.0,...,0.0,1.0,3.0,0.0,0.0,263.0,58.0,1.0,7.0,0.0
2,16.985438,23.672425,20.473448,31.349968,7.0,673.0,4.741,2,0.0,1.0,...,0.0,5.0,1.0,0.0,1.0,137.0,32.0,4.0,0.0,0.0
3,10.483405,15.921617,15.125536,40.845070,6.0,939.0,4.131,8,1.0,2.0,...,1.0,8.0,5.0,0.0,2.0,131.0,40.0,4.0,0.0,0.0
4,28.530880,22.070744,28.445099,21.702284,9.0,2380.0,5.706,4,3.0,1.0,...,0.0,8.0,6.0,0.0,8.0,760.0,86.0,11.0,2.0,0.0


In [14]:
### 데이터 정규화

std_scaler = StandardScaler()
df_std = std_scaler.fit_transform(df)
df_std = pd.DataFrame(df_std)
df_std.columns = depend_list + selected_var_name


In [15]:
df_std

,solo_all_ratio,solo_20_ratio,solo_30_ratio,solo_67_ratio,금융기관,사업체수,지가상승률,노인복지시설,안심귀갓길,대학교,...,병원,세탁업,약국,영화상영관,이용업,일반음식점,카페,편의점,쇼핑,대형마트
0,0.499530,0.040769,0.223449,-0.477480,-0.482261,1.088984,-0.217220,-0.860968,-0.207472,2.702917,...,-0.250647,-0.919198,-0.822359,-0.039105,-0.625844,0.317805,-0.004862,-0.889469,0.533591,-0.586804
1,1.509875,0.554173,-0.269980,-0.523687,-0.054791,-0.683339,-0.316483,-0.947147,-0.207472,1.055275,...,-0.871718,-1.214063,-1.213515,-0.653095,-1.293834,-0.908102,-0.625017,-1.214224,0.533591,-0.586804
2,-0.258968,0.271330,-0.142979,0.063390,-0.339771,-0.672887,0.167920,-0.774789,-0.207472,1.055275,...,-0.871718,-1.045569,-1.284634,-0.653095,-1.227035,-1.097396,-0.903017,-1.133036,-1.169729,-0.586804
3,-0.904584,-0.374038,-0.898894,0.876768,-0.411016,-0.540498,-0.437583,-0.257716,0.729498,2.702917,...,-0.561183,-0.919198,-1.142396,-0.653095,-1.160236,-1.106410,-0.817478,-1.133036,-1.169729,-0.586804
4,0.887432,0.137967,0.983796,-0.763060,-0.197281,0.176693,1.125805,-0.602431,2.603439,1.055275,...,-0.871718,-0.919198,-1.106836,-0.653095,-0.759442,-0.161440,-0.325632,-0.943595,-0.683066,-0.586804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,0.909436,-0.231640,0.359401,-0.149244,2.652522,-0.623615,-0.406812,-0.947147,-0.207472,1.055275,...,-0.871718,-1.171940,-1.249075,-0.653095,-1.160236,-1.004251,-0.678478,-1.133036,0.533591,-0.586804
416,2.917902,0.191412,-0.506404,-0.517207,0.230189,3.929878,-0.267844,-0.774789,-0.207472,1.055275,...,-0.871718,-0.666456,0.635586,0.574884,1.110932,2.356476,1.203369,-0.429399,-0.439735,-0.586804
417,-0.575355,0.159535,0.415880,-0.117765,0.087699,0.064212,1.501019,-0.602431,-0.207472,1.055275,...,1.612568,0.428758,0.279990,-0.653095,0.108946,0.433485,1.214061,0.653119,-0.683066,-0.586804
418,0.568332,0.432801,-0.373336,-0.211814,-0.624751,-0.170206,-0.480266,0.862609,3.540409,-0.592366,...,1.612568,1.229108,0.884504,0.267889,1.444927,0.711417,-0.069016,0.328364,-0.196403,0.963244


In [16]:
### 분석 - OLS

std_result = pd.DataFrame()
for idx, depend in enumerate(depend_list):
    X = sm.add_constant(df_std[selected_var_name])

    # Fit the OLS model
    model = sm.OLS(df_std[depend], X)
    result = model.fit()

    summary = result.summary()

    temp = summary.tables[1].as_csv()

    ols_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
    ols_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']
    ols_result = ols_result[['var', 'coef', 'p']]
    ols_result['p'] = ols_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
    ols_result['coef'] = ols_result['coef'].apply(lambda x: round(x, 3))
    ols_result.columns = [f'var', f'c_{depend[5:7]}', f'p_{depend[5:7]}']
    ols_result['var'] = ols_result['var'].str.strip()

    globals()[f'ols_{depend}'] = ols_result

    if idx == 0:
        std_result = ols_result
    else:
        std_result = std_result.merge(ols_result,
                                      how = 'left',
                                      on = 'var')

In [17]:
std_result

,var,c_al,p_al,c_20,p_20,c_30,p_30,c_67,p_67
0,const,0.000,,-0.000,,0.000,,0.000,
1,금융기관,-0.003,,-0.101,,0.116,+,-0.032,
2,사업체수,0.214,+,0.058,,0.276,+,-0.195,+
3,지가상승률,-0.083,,-0.068,,0.108,+,-0.045,
4,노인복지시설,0.049,,-0.135,,-0.362,+,0.268,+
5,안심귀갓길,0.264,+,0.176,+,0.087,+,-0.160,+
6,대학교,0.027,,0.137,+,-0.057,,-0.016,
7,반려동물업,-0.762,+,-0.786,+,0.055,,0.502,+
8,미용업,-0.254,,0.075,,0.198,,-0.010,
9,병원,0.060,,-0.068,,-0.034,,0.017,


In [18]:
std_result.to_csv(f'{final_result}ols_all.csv', index = False)

In [19]:
std_result.to_excel(f'{final_result}ols_all.xlsx', index = False)

In [20]:
#  out_result

## 정규화 이전

In [21]:
# final_result = pd.DataFrame()

# for depend in depend_list:
#     X = sm.add_constant(df[selected_var])

#     # Fit the OLS model
#     model = sm.OLS(df[depend], X)
#     result = model.fit()

#     summary = result.summary()

#     temp = summary.tables[1].as_csv()
#     temp2 = pd.read_csv(StringIO(temp), skipinitialspace=True)
#     temp2.columns = ['var', 'coef', 'std', 't', 'pvalue', 'confi25', 'conf97']
#     temp2 = temp2[['var', 'coef', 'pvalue']]
#     temp2.columns = [f'var', f'coef_{depend}', f'pvalue_{depend}']

#     if depend == 'solo_all':
#         final_result = temp2
#     else:
#         final_result = final_result.merge(temp2,
#                             how = 'left',
#                             on = 'var')


In [22]:
# final_result

In [23]:
# final_result.to_csv('/content/drive/MyDrive/new_project1/final_result/first200.csv', index = False)

## 컬럼명을 영어로 바꾸는 준비
- 전진선택법, 후진제거법은 변수명 영어로

In [24]:
df2_std = df_std.copy()

new_columns = ['var{}'.format(i) for i in range(1,19)]
df2_std.columns = depend_list + new_columns

In [25]:
df2_std.columns

Index(['solo_all_ratio', 'solo_20_ratio', 'solo_30_ratio', 'solo_67_ratio',
       'var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9',
       'var10', 'var11', 'var12', 'var13', 'var14', 'var15', 'var16', 'var17',
       'var18'],
      dtype='object')

In [26]:
df_std.columns

Index(['solo_all_ratio', 'solo_20_ratio', 'solo_30_ratio', 'solo_67_ratio',
       '금융기관', '사업체수', '지가상승률', '노인복지시설', '안심귀갓길', '대학교', '반려동물업', '미용업', '병원',
       '세탁업', '약국', '영화상영관', '이용업', '일반음식점', '카페', '편의점', '쇼핑', '대형마트'],
      dtype='object')

In [27]:
col_dict = {}

for i in range(len(df_std.columns)):
    col_dict[df2_std.columns[i]] = df_std.columns[i]
    col_dict[df_std.columns[i]] = df2_std.columns[i]

col_dict.update({'Intercept' : '상수',
                 '상수' : 'intercept',
                 'const' : '상수',
                 '상수' : 'const'})

## 전진선택법

In [28]:
### 전진선택법 함수

def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response, ' + '.join(selected + [candidate]))
            score = sm.OLS.from_formula(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        else:
            break
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = sm.OLS.from_formula(formula, data).fit()
    return model


In [29]:
fwd_result = pd.DataFrame()

for idx, depend in enumerate(depend_list):
    drop_list = [item for item in depend_list if item != depend]

    model = forward_selection(df2_std.drop(columns = drop_list), depend)
    summary = model.summary()
    temp = pd.DataFrame(summary.tables[1])

    temp = summary.tables[1].as_csv()
    temp_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
    temp_result.columns = ['var', 'coef', 'std', 't', 'pvalue', 'confi25', 'conf97']

    temp_result['var'] = temp_result['var'].str.strip()
    temp_result['coef'] = temp_result['coef'].apply(lambda x: round(x, 3))
    temp_result['var_mapped'] = temp_result['var'].apply(lambda x: col_dict[x])
    temp_result = temp_result[['var', 'var_mapped', 'coef', 'pvalue' ]]
    temp_result['pvalue'] = temp_result['pvalue'].apply(lambda x: '+' if x < 0.05 else ' ')
    temp_result['var_mapped'] = temp_result['var_mapped'].str.strip()
    temp_result.columns = [f'var2', f'var',f'c_{depend[5:7]}',f'p_{depend[5:7]}']



    globals()[f'fwd_{depend}'] = temp_result

    if idx == 0:
        fwd_result = temp_result
    else:
        fwd_result = fwd_result.merge(temp_result,
                            how = 'outer',
                            on = ['var', 'var2'])

In [30]:
fwd_result

,var2,var,c_al,p_al,c_20,p_20,c_30,p_30,c_67,p_67
0,Intercept,상수,0.000,,-0.000,,0.000,,0.000,
1,var2,사업체수,0.233,+,0.062,,0.260,+,-0.203,+
2,var5,안심귀갓길,0.260,+,0.181,+,0.089,+,-0.164,+
3,var17,쇼핑,0.117,+,NaN,NaN,NaN,NaN,NaN,NaN
4,var18,대형마트,-0.140,+,-0.092,+,NaN,NaN,0.095,+
5,var7,반려동물업,-0.865,+,-0.815,+,NaN,NaN,0.562,+
6,var14,일반음식점,0.392,+,NaN,NaN,0.413,+,-0.331,+
7,var10,세탁업,0.086,,NaN,NaN,0.539,+,-0.311,
8,var3,지가상승률,-0.095,+,-0.071,,0.098,+,NaN,NaN
9,var13,이용업,0.163,,NaN,NaN,-0.397,+,NaN,NaN


In [31]:
fwd_result.to_csv(f'{final_result}fwd.csv', index = False)

In [32]:
fwd_result.to_excel(f'{final_result}fwd.xlsx', index = False)

## 후진제거법

In [33]:
def backward_elimination(data, response, significance_level=0.05):
    features = data.columns.tolist()
    features.remove(response)

    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])  # Add a constant term for the intercept
        model = sm.OLS(data[response], features_with_constant).fit()
        p_values = model.pvalues.iloc[1:]  # Get p-values for the features only
        max_p_value = p_values.max()  # Find the feature with the maximum p-value
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break

    model = sm.OLS(data[response], sm.add_constant(data[features])).fit()
    return model



In [34]:
bwd_result = pd.DataFrame()

for idx, depend in enumerate(depend_list):

    drop_list = [item for item in depend_list if item != depend]

    model = backward_elimination(df2_std.drop(columns = drop_list), depend)
    summary = model.summary()
    temp_bwd = pd.DataFrame(summary.tables[1])

    temp_bwd = summary.tables[1].as_csv()
    temp_bwd = pd.read_csv(StringIO(temp_bwd), skipinitialspace=True)
    temp_bwd.columns = ['var', 'coef', 'std', 't', 'pvalue', 'confi25', 'conf97']

    temp_bwd['var'] = temp_bwd['var'].str.strip()
    temp_bwd['coef'] = temp_bwd['coef'].apply(lambda x: round(x, 3))
    temp_bwd['var_mapped'] = temp_bwd['var'].apply(lambda x: col_dict[x])
    temp_bwd = temp_bwd[['var', 'var_mapped', 'coef', 'pvalue' ]]
    temp_bwd['var_mapped'] = temp_bwd['var_mapped'].str.strip()
    temp_bwd['pvalue'] = temp_bwd['pvalue'].apply(lambda x: '+' if x < 0.05 else ' ')
    temp_bwd.columns = [f'var2', f'var',f'c_{depend[5:7]}', f'p_{depend[5:7]}']


    globals()[f'bwd_{depend}'] = temp_bwd

    globals()[f'bwd_{depend}']

    if idx == 0:
        bwd_result = temp_bwd
    else:
        bwd_result = bwd_result.merge(temp_bwd,
                            how = 'outer',
                            on = ['var', 'var2'])

In [35]:
bwd_result

,var2,var,c_al,p_al,c_20,p_20,c_30,p_30,c_67,p_67
0,const,상수,0.000,,-0.000,,0.000,,0.000,
1,var2,사업체수,0.220,+,NaN,NaN,0.257,+,-0.159,+
2,var3,지가상승률,-0.094,+,NaN,NaN,0.100,+,NaN,NaN
3,var5,안심귀갓길,0.262,+,0.192,+,0.092,+,-0.164,+
4,var7,반려동물업,-0.856,+,-0.783,+,NaN,NaN,0.527,+
5,var13,이용업,0.183,+,NaN,NaN,-0.343,+,NaN,NaN
6,var14,일반음식점,0.389,+,NaN,NaN,0.226,+,-0.365,+
7,var16,편의점,0.308,+,0.509,+,NaN,NaN,-0.458,+
8,var17,쇼핑,0.118,+,NaN,NaN,NaN,NaN,NaN,NaN
9,var18,대형마트,-0.140,+,-0.089,+,NaN,NaN,0.084,+


In [36]:
bwd_result.to_csv(f'{final_result}bwd.csv', index = False)

In [37]:
bwd_result.to_excel(f'{final_result}bwd.xlsx', index = False)

## 각 연령대별 결과 통합

In [38]:
for depend in depend_list:

    ols = globals()[f'ols_{depend}']
    fwd = globals()[f'fwd_{depend}']
    bwd = globals()[f'bwd_{depend}']

    ols['var'] = ols['var'].apply(lambda x: x if x != 'const' else '상수')

    temp = ols.merge(fwd.drop(columns = 'var2'),
                  how = 'outer',
                  on = 'var',
                  suffixes=('_ols', '_fwd'))
    temp = temp.merge(bwd.drop(columns = 'var2'),
                  how = 'outer',
                  on = 'var',
                  suffixes=('_fwd', '_bwd'))

    temp = temp.sort_values(by=[f'c_{depend[5:7]}_fwd', f'p_{depend[5:7]}'])

    globals()[f'result_{depend}']= temp

    globals()[f'result_{depend}'].to_csv(f'{final_result}_{depend}.csv', index = False)
    globals()[f'result_{depend}'].to_excel(f'{final_result}_{depend}.xlsx', index = False)

In [39]:
result_solo_67_ratio

,var,c_67_ols,p_67_ols,c_67_fwd,p_67_fwd,c_67,p_67
14,일반음식점,-0.404,+,-0.331,+,-0.365,+
10,세탁업,-0.325,,-0.311,,NaN,NaN
16,편의점,-0.270,,-0.229,,-0.458,+
2,사업체수,-0.195,+,-0.203,+,-0.159,+
5,안심귀갓길,-0.160,+,-0.164,+,-0.164,+
1,금융기관,-0.032,,-0.059,,NaN,NaN
0,상수,0.000,,0.000,,0.000,
18,대형마트,0.084,,0.095,+,0.084,+
4,노인복지시설,0.268,+,0.307,+,0.286,+
7,반려동물업,0.502,+,0.562,+,0.527,+
